# Video Augmentation

참고 : 

- https://github.com/Rafalves883/Video-Augmentation
- https://www.tensorflow.org/tutorials/images/data_augmentation?hl=ko

# Step

1. video frames stored as images ndarrys
2. image preprocessing (mostly, image rotation)
3. images into video

## 실행
vid_aug(증강할 영상 경로, 생성된 영상 이름과 경로) 실행해주면 됨.

## 변경
random degree 범위는 random_rotation1() 에서 random.uniform(-%,%) 으로 설정하면 됨.

In [1]:
import random
from scipy import ndarray
import skimage as sk
from skimage import transform
from skimage import util
import cv2
from skimage import data
import warnings
from skimage import filters
from skimage import color
from skimage import exposure
from skimage.transform import rotate
from skimage.transform import warp
from skimage.transform import ProjectiveTransform
from skimage.transform import AffineTransform
import math
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers
import tensorflow_addons as tfa

In [2]:
#turns video into frames
#frame rate 조절해서 프레임 수 바꾸기(30으로 설정한 이유는 fps값이 29.9이기 때문)
def get_frames(video_path, frameRate=1/29):
    vidcap = cv2.VideoCapture(video_path)
    def getFrame(sec):
        vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
        hasFrames,image = vidcap.read()
        return hasFrames, image
    images= []
    sec = 0
    count=1
    success = getFrame(sec)
    while success:
        count = count + 1
        sec = sec + frameRate
        sec = round(sec, 2)
        success, imagem = getFrame(sec)
        images.append(imagem)
    return images

In [3]:
def random_rotation1(images):
    def random_rotation(image_array, random_degree):
        # pick a random degree of rotation between 25% on the left and 25% on the right
        return sk.transform.rotate(image_array, random_degree)
    img_arroi=[]
    random_degree = random.uniform(-25, 25)
    for i in range(len(images)-1):
        image_with_random_noise = random_rotation(images[i],random_degree)
        image_with_random_noise =(image_with_random_noise *255).astype(np.uint8)
        img_arroi.append(image_with_random_noise)
    return img_arroi

In [9]:
def turn_video(video_name, frames):
    video_height, video_width, l = frames[0].shape
    
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    out = cv2.VideoWriter(video_name, fourcc,29, (int(video_width), int(video_height)))
    
    for frame in frames:
        out.write(frame)
    out.release()

In [12]:
def vid_aug(origin_video_path, output_video_path):
    file = origin_video_path
    frames = get_frames(file)
    rot_frames = random_rotation1(frames)
    turn_vid = turn_video(output_video_path,rot_frames)
    return turn_vid
    

In [14]:
vid_aug('origin/abnormal_gait1.mp4','outputs/rand2.avi')

## 사용가능한 다른 aug 기법

- flip
- degree custom rotate
- invert
- add(모든 픽셀 값에 특정 값 더하기)
- pepper (black point)
- salt (white point)
- speed

In [12]:
#원래는 param 이 clip이었는데 images로 바꾸어놓았음.
def random_noise_v(images):
    from skimage.util import random_noise
    img_array=[]
    for i in range(len(images)-1):
        image_with_random_noise = random_noise(images[i])
        img_array.append(image_with_random_noise)
    return img_array

In [19]:
def gaussian_b(images, intensity=0.75, depth=1):
    def gaussian(X, intensity=0.75, depth=1):
        indices_gaussian = np.random.choice(
            X.shape[0], math.ceil(X.shape[0] * depth), replace=False)
        X_=[]
        for k in indices_gaussian:
            sigma_=uniform(1-intensity,intensity)
            X_.append(filters.gaussian(X[k], sigma=sigma_, multichannel=True))
        return np.asarray(X_)
    img_arrai=[]
    for i in range(len(images)-1):
        image_with_random_noise = gaussian(images[i],intensity,depth)
        img_arrai.append(image_with_random_noise)
    return img_arrai

In [98]:
#degree custom rotate 함수 만듦.
def rotation(images, degree):
    img_arr = []
    for i in range(len(images)):
        rot_img = sk.transform.rotate(images[i],degree)
        img_arr.append(rot_img)
    return img_arr

In [10]:
def random_rotation1(images):
    def random_rotation(image_array, random_degree):
        # pick a random degree of rotation between 25% on the left and 25% on the right
        return sk.transform.rotate(image_array, random_degree)
    img_arroi=[]
    random_degree = random.uniform(-25, 25)
    for i in range(len(images)-1):
        image_with_random_noise = random_rotation(images[i],random_degree)
        img_arroi.append(image_with_random_noise)
    return img_arroi

In [7]:
#horizontal flip de video
def flip(images):
    def horizontal_flip(image_array):
        # horizontal flip doesn't need skimage, it's easy as flipping the image array of pixels !
        return image_array[:, ::-1]
    img_arrei=[]
    for i in range(len(images)-1):
        image_with_random_noise = horizontal_flip(images[i])
        img_arrei.append(image_with_random_noise)
    return img_arrei

In [8]:
#invert video color
def invert(images):
    img_arrui=[]
    for i in range(len(images)-1):
        image_with_random_noise = np.invert(images[i])
        img_arrui.append(image_with_random_noise)
    return img_arrui

In [9]:
#add value to all pixels
def add(images, value):
    data_final = []
    for i in range(len(images)-1):
        image = images[i].astype(np.int32)
        image += value
        image = np.where(image > 255, 255, image)
        image = np.where(image < 0, 0, image)
        image = image.astype(np.uint8)
        data_final.append(image.astype(np.uint8))
    return data_final

In [10]:
#apply random black points on the video
def pepper(images):
    dota_final = []
    for i in range(len(images)-1):
        img = images[i].astype(np.float)
        img_shape = img.shape
        noise = np.random.randint(30, size=img_shape)
        img = np.where(noise == 0, 0, img)
        dota_final.append(img.astype(np.uint8))
    return dota_final

In [18]:
#apply random white points on the video
def salt(images):
    deta_final = []
    for i in range(len(images)-1):
        img = images[i].astype(np.float64)
        img_shape = img.shape
        noise = np.random.randint(30, size=img_shape)
        img = np.where(noise == 0, 255, img)
        deta_final.append(img.astype(np.uint8))
    return deta_final

In [12]:
#if 0<ratio<1, speed up video, else 1<ratio<infinity, slows up the video
def changesample(clip, ratio):
        nb_return_frame = np.floor(ratio * len(clip))
        return_ind = [int(i) for i in np.linspace(1, len(clip), num=nb_return_frame)]
        return [clip[i-1] for i in return_ind]